In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [ ]:
# Define cubic SE6, find and classify the lines and compute the 45 tritangent planes

In [3]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE6 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f: -c*(5*c+e)/(c+e)})

In [ ]:
# Find simmetries

In [4]:
%time adm_SE6 = SE6.find_admissible_projectivities()
len(adm_SE6)

CPU times: user 374 ms, sys: 87.3 ms, total: 461 ms
Wall time: 1.1 s


48

In [5]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)
len(simm_SE6)

CPU times: user 180 ms, sys: 52.3 ms, total: 232 ms
Wall time: 285 ms


24

In [ ]:
# Study how the simmetries permute the Eckardt points

In [6]:
Eck_points_perms = []
for proj in simm_SE6:
    Eck_points_perms.append(Permutation(SE6.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(24, 'S4')

In [ ]:
# We want to find what elements are permuted by the group S4
# These are the lines through 2 Eckardt points that are not contained in SE6

In [7]:
indices = []
for gen in Eck_points_perms_group.minimal_generating_set():
    indices.append(Eck_points_perms.index(gen))
gen1 = simm_SE6[indices[0]]
gen2 = simm_SE6[indices[1]]

In [9]:
lines_through_2_Eck_points = []
lines_plucker = [line.plucker for line in SE6.lines]
for i in range(len(eck)):
    point1 = eck[i]
    for j in range(i+1, len(eck)):
        point2 = eck[j]
        new_line_plucker = Point(matrix([point1, point2]).minors(2))
        #check that we get a new line that does not belong to SE6
        if new_line_plucker not in lines_plucker and new_line_plucker not in [line.plucker for line in lines_through_2_Eck_points]:
            planes = get_two_planes_containing_line(new_line_plucker)
            lines_through_2_Eck_points.append(Line(planes, [point1, point2], new_line_plucker))

In [10]:
#check that the 4 lines are actually permuted by the simmetry group
for line in lines_through_2_Eck_points:
    print([line.apply_proj(gen1) in lines_through_2_Eck_points, line.apply_proj(gen2) in lines_through_2_Eck_points])

[True, True]
[True, True]
[True, True]
[True, True]


In [ ]:
# Study how the simmetries permute the 27 lines

In [11]:
lines_perms = []
for simm in simm_SE6:
    lines_perms.append(Permutation(SE6.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(24, 'S4')

In [13]:
[el.order() for el in lines_perms_group.normal_subgroups()]

[24, 12, 4, 1]

In [ ]:
# Find possible conditions to obtain a subfamily with a larger simmetry group

In [14]:
%time SE6.find_conditions_for_subfamilies(adm_SE6, simm_SE6)

CPU times: user 473 ms, sys: 8.19 ms, total: 481 ms
Wall time: 503 ms


[]

In [8]:
eck_labels = SE6.eckardt_points_labels
eck = SE6.eckardt_points
for i in range(4):
    for j in range(i+1, 5):
        for k in range(j+1, 6):
            if matrix([eck[i], eck[j], eck[k]]).minors(3) == [0,0,0,0]:
                print(eck_labels[i], eck_labels[j], eck_labels[k])

['E1', 'G4', 'F14'] ['E2', 'G1', 'F12'] ['E4', 'G2', 'F24']
['E1', 'G4', 'F14'] ['E2', 'G3', 'F23'] ['F13', 'F24', 'F56']
['E2', 'G1', 'F12'] ['E3', 'G4', 'F34'] ['F13', 'F24', 'F56']
['E2', 'G3', 'F23'] ['E3', 'G4', 'F34'] ['E4', 'G2', 'F24']
